# Real Estate EDA

### Imports (Pandas - Python Data Analysis Library, Numbpy - Array Proessing Package, Seaborn - Statistical Data Visualisation, Matplotlib - Static Vizualization Library)

In [15]:
# Standard Import Structure for EDA;
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Importing Requests for use in API calls.
import requests


# Import Standard Packages for Date and Time;
from datetime import datetime, timedelta

# Import Beautiful Soup, Selenium, for Scraping
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Import packages to help with parsing Information
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext


### Setup and Initialization

In [16]:
# Set Maximum Displayed Columns to None; All Columns Listed
pd.options.display.max_columns = None;

# Set MatPlotLib to Display Elements
%matplotlib inline

# Import Cities in CSV file for 10 most and least expenive north carolina cities.csv
cities_df = pd.read_csv("cities.csv",header=0)

print(cities_df)

# Initialize Variables to be Read into.
address = ''
price = 0
beds = 0
details = 0
home_type = ''
last_updated = ''
brokerage = ''
link = ''


        Expensive   Affordable
0            Elon         Eden
1           Boone        Ayden
2         Woodfin       Maiden
3        Carrboro       Hamlet
4       Asheville       Newton
5       Pineville     Sawmills
6      Wilmington     ArchDale
7   Morehead City   Rockingham
8  Black Mountain  Winterville
9  Hendersonville  Gibsonville


In [17]:
# Insert Headers for various websites to pull information specifically.
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

# Checking for Sale using Zillow.com
# Using 20 Cities we use 20 URLs
# Using loop to collect information from csv to pass to beautiful soup
for i in range(9):
    for j in range(2):
        # Get City from cities_df
        city = cities_df.iat[i,j]
        print(city)

        # Desired City
        location = city0+',-NC_rb/'

        # URL coresponding to Zillow.com
        url = 'https://www.zillow.com/homes/for_sale/'+location    
        # Use Get Request
        r = requests.get(url, headers=req_headers)

        soup = BeautifulSoup(r0.text, 'html.parser')

        house_df = pd.DataFrame()
        

Elon
Eden
Boone
Ayden
Woodfin
Maiden
Carrboro
Hamlet
Asheville
Newton
Pineville
Sawmills
Wilmington
ArchDale
Morehead City
Rockingham
Black Mountain
Winterville
